In [ ]:
import marimo as mo
from problem_models import graph_colouring

In [ ]:
import inspect


def show_method(module, class_name=None, func_name=""):
    if class_name:
        cls = getattr(module, class_name, None)
        if cls:
            if func_name:
                if func_name == "__init__":
                    # Get class signature
                    class_signature = (
                        f"class {class_name}{inspect.signature(cls)}"
                    )

                    # Get all dunder methods
                    dunder_methods = {
                        name: method
                        for name, method in vars(cls).items()
                        if callable(method)
                        and name.startswith("__")
                        and name.endswith("__")
                    }

                    # Get source code for dunder methods
                    dunder_sources = []
                    for name, method in dunder_methods.items():
                        try:
                            dunder_sources.append(inspect.getsource(method))
                        except TypeError:
                            dunder_sources.append(
                                f"# Source not available for {name}"
                            )

                    return (
                        f"```python\n{class_signature}\n"
                        + "\n".join(dunder_sources)
                        + "\n```\n"
                    )
                else:
                    func = getattr(cls, func_name, None)
            else:
                # Show class definition with all methods
                class_source = inspect.getsource(cls)
                return f"```python\n{class_source}\n```\n"
        else:
            return f"Class '{class_name}' not found in module."
    else:
        func = getattr(module, func_name, None)

    return (
        f"```python\n{inspect.getsource(func)}\n```\n"
        if func
        else f"Method or function '{func_name}' not found."
    )

# ROAR-NET API Specification examples


The following is an example of how the [ROAR-NET API Specification]() can be used to model the Graph-Colouring problem.


## **Usecase: Graph colouring**

The graph colouring problem can be formally defined as follows:

Given an undirected graph \( G = (V,E) \), where \( V \) is the set of vertices and \( E \) is the set of edges.

A **proper $k$-colouring** of the graph \( G \) is a mapping \( C: V \rightarrow K \), where $K = \{1,2,\dots,k\}$,
such that for every edge \( (u, v) \in E \), the condition $c(u) \neq c(v)$ holds.

A **color class** corresponding to a color \( i \in K \) is the set of all vertices assigned the same color. Formally, $K_i = \{ v \in V \;|\; c(v) = i \}.$ The set of **uncolored vertices** is defined as: $V' = V  \setminus \bigcup_{i \in K} K_i.$

The **conflict set** \( E' \) is the set of all edges where the vertices connected by the edge have the same color. This can be defined as:   $E' = \bigcup_{i \in K} \{(u,v) \in E \;|\; u,v \in K_i\}$ or equivalently, $E' = \{(u,v) \in E \;|\; c(u) = c(v)\}.$

A lower and upper bound for a graph $G$ can be set as: $\omega(G) < |K| < \Delta(G)+1$


## **Strategy**

This model uses a k-variable complete coloring strategy, where the number of colors used is controlled during the optimization process. It employs the same evaluation function: $f(s) = - \sum_{i\in K}^{} |K_i|^2 + (|E'| + |V'|)$ as described by Johnson et al. (1991) in their study on simulated annealing for graph coloring and number partitioning. They demonstrated that a penalty function approach was not the best performer when using simulated annealing. However, it was chosen because the specification does not implicitly support evaluation methods, only an objective function. The model will also have the flexibility to use other heuristics to see if the performance improves compared to the results presented by Johnson et al. (1991).

<br/>
<br/>

In [ ]:
mo.vstack(
    [
        mo.md(
            r"""

        ### **Implementation: API-Operations**

        The API is implemented as a pyhton package with an Opertaions module. This module implements the types and abstract classes to work as interfaces to be implemented in problem-models:
         """
        ),
        mo.accordion(
            {
                "View code:": mo.md(
                    r"""
                        ```python

                        class ProblemInterface(ABC):
                            @abstractmethod
                            def construction_neighbourhood(self: Problem) -> Neighbourhood:
                                ...

                            @abstractmethod
                            def destruction_neighbourhood(self: Problem) -> Neighbourhood:
                                ...

                            @abstractmethod
                            def local_neighbourhood(self: Problem) -> Neighbourhood:
                                ...

                            @abstractmethod
                            def random_solution(self: Problem) -> Solution:
                                ...

                            @abstractmethod
                            def empty_solution(self: Problem) -> Solution:
                                ...

                            @abstractmethod
                            def heuristic_solution(self: Problem) -> Optional[Solution]:
                                ...


                        class SolutionInterface(ABC):

                            @abstractmethod
                            def objective_value(self) -> float:
                                ...

                            @abstractmethod
                            def copy_solution(self: Solution) -> Solution:
                                ...

                            @abstractmethod
                            def is_feasible(self: Solution) -> bool:
                                ...

                            @abstractmethod
                            def objective_value_increment(self: Solution, move: Move) -> Optional[float]:
                                ...

                            @abstractmethod
                            def apply(self: Solution, move: Move) -> Solution:
                                ...

                            @abstractmethod
                            def get_num_colours(self: Solution) -> int:
                                ...

                        class NeighbourhoodInterface(ABC):
                            @abstractmethod
                            def moves(self: Neighbourhood, solution: Solution) -> Iterable[Move]:
                                ...

                            @abstractmethod
                            def random_move(self: Neighbourhood, solution: Solution) -> Optional[Move]:
                                ...

                            @abstractmethod
                            def random_moves_without_replacement(self: Neighbourhood, solution: Solution) -> Optional[Move]:
                                ...


                        class MoveInterface(ABC):
                            @abstractmethod
                            def invert(self: Move) -> Move:
                                raise NotImplementedError

                        Solution = TypeVar('Solution', bound=SolutionInterface)
                        Move = TypeVar('Move', bound=MoveInterface)
                        Neighbourhood = TypeVar('Neighbourhood', bound=NeighbourhoodInterface)
                        Problem = TypeVar('Problem', bound=ProblemInterface)

                        ```

                        """
                )
            }
        ),
        mo.md(
            r"""
            All operations are implemented as function calls, to have the same function signatures as the specification:
            """
        ),
        mo.accordion(
            {
                "View code:": mo.md(
                    r"""
                    ```python
                    def apply(move: Move, solution: Solution) -> Solution:
                     return solution.apply(move)

                    def construction_neighbourhood(problem: Problem) -> Neighbourhood:
                     return problem.construction_neighbourhood()

                    def destruction_neighbourhood(problem: Problem) -> Neighbourhood:
                     return problem.destruction_neighbourhood()

                    def local_neighbourhood(problem: Problem) -> Neighbourhood:
                     return problem.local_neighbourhood()

                    def empty_solution(problem: Problem) -> Solution:
                     return problem.empty_solution()

                    def random_solution(problem: Problem) -> Solution:
                     return problem.random_solution()

                    def heuristic_solution(problem: Problem) -> Optional[Solution]:
                     return problem.heuristic_solution()

                    def copy_solution(solution: Solution) -> Solution:
                     return solution.copy_solution()

                    def invert(move: Move) -> Move:
                     return move.invert()

                    def is_feasible(solution: Solution) -> bool:
                     return solution.is_feasible()

                    def lower_bound_increment(move: Move, solution: Solution) -> Optional[float]:
                     return solution.lower_bound_increment()

                    def lower_bound(solution: Solution) -> Optional[float]:
                     return solution.lower_bound()

                    def moves(neighbourhood: Neighbourhood, solution: Solution) -> Iterable[Move]:
                     return neighbourhood.moves(solution)

                    def objective_value_increment(move: Move, solution: Solution) -> Optional[float]:
                     return solution.objective_value_increment(move)

                    def objective_value(solution: Solution) -> Optional[float]:
                     return solution.objective_value()

                    def random_move(neighbourhood: Neighbourhood, solution: Solution) -> Optional[Move]:
                     return neighbourhood.random_move(solution)

                    def random_moves_without_replacement(neighbourhood: Neighbourhood, solution: Solution) -> Optional[Move]:
                     return neighbourhood.random_moves_without_replacement(solution)

                    def get_num_colours(solution: Solution) -> int:
                     return solution.get_num_colours()
                    ```
                    """
                )
            }
        ),
    ]
)

Implementation: API-Operations 
 The API is implemented as a pyhton package with an Opertaions module. This module implements the types and abstract classes to work as interfaces to be implemented in problem-models: class ProblemInterface ( ABC ): 
 @abstractmethod 
 def construction_neighbourhood ( self : Problem ) -> Neighbourhood : 
 ... 

 @abstractmethod 
 def destruction_neighbourhood ( self : Problem ) -> Neighbourhood : 
 ... 

 @abstractmethod 
 def local_neighbourhood ( self : Problem ) -> Neighbourhood : 
 ... 

 @abstractmethod 
 def random_solution ( self : Problem ) -> Solution : 
 ... 

 @abstractmethod 
 def empty_solution ( self : Problem ) -> Solution : 
 ... 

 @abstractmethod 
 def heuristic_solution ( self : Problem ) -> Optional [ Solution ]: 
 ... 


 class SolutionInterface ( ABC ): 

 @abstractmethod 
 def objective_value ( self ) -> float : 
 ... 

 @abstractmethod 
 def copy_solution ( self : Solution ) -> Solution : 
 ... 

 @abstractmethod 
 def is_feasible ( self : Solution ) -> bool : 
 ... 

 @abstractmethod 
 def objective_value_increment ( self : Solution , move : Move ) -> Optional [ float ]: 
 ... 

 @abstractmethod 
 def apply ( self : Solution , move : Move ) -> Solution : 
 ... 

 @abstractmethod 
 def get_num_colours ( self : Solution ) -> int : 
 ... 

 class NeighbourhoodInterface ( ABC ): 
 @abstractmethod 
 def moves ( self : Neighbourhood , solution : Solution ) -> Iterable [ Move ]: 
 ... 

 @abstractmethod 
 def random_move ( self : Neighbourhood , solution : Solution ) -> Optional [ Move ]: 
 ... 

 @abstractmethod 
 def random_moves_without_replacement ( self : Neighbourhood , solution : Solution ) -> Optional [ Move ]: 
 ... 


 class MoveInterface ( ABC ): 
 @abstractmethod 
 def invert ( self : Move ) -> Move : 
 raise NotImplementedError 

 Solution = TypeVar ( 'Solution' , bound = SolutionInterface ) 
 Move = TypeVar ( 'Move' , bound = MoveInterface ) 
 Neighbourhood = TypeVar ( 'Neighbourhood' , bound = NeighbourhoodInterface ) 
 Problem = TypeVar ( 'Problem' , bound = ProblemInterface ) 
 All operations are implemented as function calls, to have the same function signatures as the specification: def apply ( move : Move , solution : Solution ) -> Solution : 
 return solution . apply ( move ) 

 def construction_neighbourhood ( problem : Problem ) -> Neighbourhood : 
 return problem . construction_neighbourhood () 

 def destruction_neighbourhood ( problem : Problem ) -> Neighbourhood : 
 return problem . destruction_neighbourhood () 

 def local_neighbourhood ( problem : Problem ) -> Neighbourhood : 
 return problem . local_neighbourhood () 

 def empty_solution ( problem : Problem ) -> Solution : 
 return problem . empty_solution () 

 def random_solution ( problem : Problem ) -> Solution : 
 return problem . random_solution () 

 def heuristic_solution ( problem : Problem ) -> Optional [ Solution ]: 
 return problem . heuristic_solution () 

 def copy_solution ( solution : Solution ) -> Solution : 
 return solution . copy_solution () 

 def invert ( move : Move ) -> Move : 
 return move . invert () 

 def is_feasible ( solution : Solution ) -> bool : 
 return solution . is_feasible () 

 def lower_bound_increment ( move : Move , solution : Solution ) -> Optional [ float ]: 
 return solution . lower_bound_increment () 

 def lower_bound ( solution : Solution ) -> Optional [ float ]: 
 return solution . lower_bound () 

 def moves ( neighbourhood : Neighbourhood , solution : Solution ) -> Iterable [ Move ]: 
 return neighbourhood . moves ( solution ) 

 def objective_value_increment ( move : Move , solution : Solution ) -> Optional [ float ]: 
 return solution . objective_value_increment ( move ) 

 def objective_value ( solution : Solution ) -> Optional [ float ]: 
 return solution . objective_value () 

 def random_move ( neighbourhood : Neighbourhood , solution : Solution ) -> Optional [ Move ]: 
 return neighbourhood . random_move ( solution ) 

 def random_moves_witho

### **Implementation: Graph colouring problem - Problem**

The problem is implemented as a class that inherits from the ProblemInterface. The class should implement all the data necessary to represent a particular instance of the Graph Colouring problem. The edges between connected vertecies are reprecented as both an adjecency matrix and adjecency list. The number of vertecies are also saved for easy refrence:

In [ ]:
mo.md(
    f"{
        show_method(
            graph_colouring,
            'Problem',
            '__init__',
        )
    }"
)

class Problem ( v : 'int' , edge_list : 'EdgeList' ) -> 'None' 
 def __init__ ( self , v : int , edge_list : EdgeList ) -> None : 
 self . vertex_count = v 
 self . adjacency_matrix = adjacency_matrix ( v , edge_list ) 
 self . adjacency_list = adjacency_list ( v , edge_list )

The **Problem class** also implements the **local_neighbourhood**, **construction_neighbourhood**, **destruction_neighbourhood** methods, which returns thier respective neighbourhood object by calling their constuctor:

To get a **Solution** instance the **Problem** class implements the following methods:

In [ ]:
mo.accordion(
    {
        "View methods: ": mo.md(
            f"{
                show_method(
                    graph_colouring,
                    'Problem',
                    'empty_solution',
                )
            }"
            f"{
                show_method(
                    graph_colouring,
                    'Problem',
                    'random_solution',
                )
            }"
            f"{
                show_method(
                    graph_colouring,
                    'Problem',
                    'heuristic_solution',
                )
            }"
        )
    }
)

def empty_solution ( self : Problem ) -> Solution : 
 """ 
 Returns an empty solution for the given problem. 

 Parameters 
 ---------- 
 self : Problem 
 The problem instance that the empty solution will be constructed for. 

 Returns 
 ------- 
 Solution 
 An empty solution for the given problem, initialized with no assignments. 

 """ 
 return Solution ( self ) 
 
 def random_solution ( self : Problem ) -> Solution : 
 """ 
 Constructs a random solution by iteratively applying random construction moves. 

 Starts with an empty solution and sequentially applies a random move 
 from the construction neighborhood until all vertices are assigned. The moves 
 are chosen from the Construction_neighbourhood of the current partial solution. 

 Parameters 
 ---------- 
 self : Problem 
 The problem instance that contains the necessary information for constructing a solution. 

 Returns 
 ------- 
 Solution 
 A fully constructed solution for the given problem. 
 """ 
 solution = Solution ( self ) 
 for v in range ( self . vertex_count ): 
 neighbourhood : Construction_neighbourhood = Construction_neighbourhood ( self ) 
 move : Move = neighbourhood . random_move ( solution ) 
 if move is None : 
 pass 
 solution . apply ( move ) 

 return solution 
 
 def heuristic_solution ( self : Problem ) -> Optional [ Solution ]: 
 """ 
 Constructs a heuristic solution by assigning the lowest colour that doesn't 
 introduce an edge conflict for each vertex. 

 This method iterates through all vertices and assigns the lowest available 
 colour (integer) that doesn't create an edge conflict with already assigned vertices. 

 Parameters 
 ---------- 
 self : Problem 
 The problem instance that the heuristic solution will be constructed for. 

 Returns 
 ------- 
 Optional[Solution] 
 A feasible solution if successful, otherwise `None` if an error occurs during 
 the colour assignment for any vertex. 

 """ 
 solution = Solution ( self ) 

 vertices = list ( range ( self . vertex_count )) 
 random . shuffle ( vertices ) 

 for v in vertices : 
 try : 
 neighbourhood = Construction_neighbourhood ( self ) 
 _moves = [ move for move in neighbourhood . moves ( solution ) if move . vertex == v and move . new_colour not in solution . adjacent_colours ( v )] 
 move = min ( _moves , key = lambda x : x . new_colour ) 
 solution . apply ( move ) 
 except ValueError : 
 logging . error ( f "Vertex { v } : Failed to assign a colour" ) 
 return None 
 return solution

### **Implementation: Graph colouring problem - Neighborhood**

The three neighbourhoods, local-, construction- and destruction-neighborhood, are implemented as seperate classes all inheriting the NeighbourhoodInterface class. They all implement their own version of the three methods, *moves*, *random_move* and *random_moves_without_replacement*.:

In [ ]:
mo.accordion(
    {
        "View Construction_neighbourhood: ": mo.md(
            f"{
                show_method(
                    graph_colouring,
                    'Construction_neighbourhood',
                )
            }"
        ),
        "View Destruction_neighbourhood: ": mo.md(
            f"{
                show_method(
                    graph_colouring,
                    'Destruction_neighbourhood',
                )
            }"
        ),
        "View Local_neighbourhood: ": mo.md(
            f"{
                show_method(
                    graph_colouring,
                    'Local_neighbourhood',
                )
            }"
        ),
    }
)

<marimo-accordion data-labels='["<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\">View Construction_neighbourhood: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\">View Destruction_neighbourhood: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\">View Local_neighbourhood: </span></span>"]' data-multiple='false'> class Construction_neighbourhood ( NeighbourhoodInterface ): 
 def __init__ ( self , problem : Problem ): 
 self . problem : Problem = problem 


 def moves ( self : Neighbourhood , solution : Solution ) -> Iterable [ Move ]: 
 """ 
 Generates all possible construction moves for the given solution. 

 Parameters 
 ---------- 
 solution : Solution 
 The current solution state, which contains the vertex assignments and colour classes. 

 Returns 
 ------- 
 Iterable[Move] 
 An iterable (set) of valid construction moves. Each move represents the assignment of a new colour 
 to an unassigned vertex. 
 """ 
 _moves = set () 
 for vertex , c in solution . vertex_colours . items (): 
 if c == 0 : 
 # Range from 1 to minimum of # of use colours and # of vertices 
 colours = min ( len ( solution . colour_classes . keys ()) + 1 , self . problem . vertex_count ) 
 adding_moves = { Move ( move_type = Move_type . CONSTRUCTION_MOVE , vertex = vertex , new_colour = new_colour ) 
 for new_colour in range ( 1 , colours + 1 )} 
 _moves . update ( adding_moves ) 
 return _moves 


 def random_move ( self : Neighbourhood , solution : Solution ) -> Optional [ Move ]: 
 """ 
 Generates a random construction move for the given solution. 

 This method randomly selects an unassigned vertex (colour 0) and assigns it a random new colour. The new colour 
 is chosen from the available colours, based on the number of used colours and the total number of vertices. 

 Parameters 
 ---------- 
 solution : Solution 
 The current solution state. 

 Returns 
 ------- 
 Optional[Move] 
 A random construction move, represented as a `Move` object, or `None` if no valid move can be generated. 
 The move consists of a vertex and a new colour assignment. 
 """ 
 vertex = choice ([ v for v , c in solution . vertex_colours . items () if c == 0 ]) 
 # Range from 1 to minimum of # of use colours and # of vertices 
 new_colour = choice ( 
 [ c for c in range ( 1 , min ( len ( solution . colour_classes . keys ()) + 1 , self . problem . vertex_count ) + 1 )]) 

 if vertex is not None and new_colour is not None : 
 return Move ( move_type = Move_type . CONSTRUCTION_MOVE , vertex = vertex , new_colour = new_colour ) 
 return None 


 def random_moves_without_replacement ( self : Neighbourhood , solution : Solution ) -> Optional [ Move ]: 
 """ 
 Returns a valid move from the move generator. It continues selecting moves without 
 replacement until a valid move is found (i.e., an unassigned vertex is assigned a new colour). If no valid moves 
 are available, it returns `None`. 

 Parameters 
 ---------- 
 solution : Solution 
 The current solution state, which contains the vertex colour assignments and move generator. 

 Returns 
 ------- 
 Optional[Move] 
 A valid construction move represented as a `Move` object, or `None` if no valid move is found. 
 """ 
 try : 
 while True : # Keep trying until we find a valid move 
 vertex , new_colour = next ( solution . construction_move_generator ) 
 if vertex and new_colour and solution . vertex_colours [ vertex ] == 0 : 
 return Move ( Move_type . CONSTRUCTION_MOVE , vertex , new_colour ) 

 except StopIteration : 
 # Generator is exhausted, return None 
 return None 
 class Destruction_neighbourhood ( NeighbourhoodInterface ): 
 def __init__ ( self , problem : Problem ): 
 self . problem : Problem = problem 


 def moves ( self : Neighbourhood , solution : Solution ) -> Iterable [ Move ]: 
 """ 
 Generates all possible destruction moves for the given solution. 

 Parameters 
 ---------- 
 solution : Solution 
 The c

### **Implementation: Graph colouring problem - Move**

A move is represented by a move type (Enum), an integer (representing the vertex), and optionally two integers representing the color of the vertex before and after the move. The move class implements the MoveInterface class and has the following methods: *Invert*, which returns the inverse of the move.

In [ ]:
mo.accordion(
    {
        "View classes: ": mo.md(
            f"{
                show_method(
                    graph_colouring,
                    'Move_type',
                )
            }"
            f"{
                show_method(
                    graph_colouring,
                    'Move',
                )
            }"
        )
    }
)

class Move_type ( Enum ): 
 CONSTRUCTION_MOVE = 1 
 DESTRUCTION_MOVE = 2 
 ONE_EXCHANGE = 3 
 
 class Move ( MoveInterface ): 
 def __init__ ( self , move_type : Move_type , vertex : int , new_colour : Optional [ int ] = None , 
 old_colour : Optional [ int ] = None ): 
 self . move_type = move_type 
 self . vertex : int = vertex 
 self . new_colour : Optional [ int ] = new_colour 
 self . old_colour : Optional [ int ] = old_colour 
 self . _cached_hash = hash (( self . move_type , self . vertex , self . old_colour , self . new_colour )) 

 def __eq__ ( self , other ): 
 if not isinstance ( other , Move ): 
 return False 
 return self . _cached_hash == other . _cached_hash 

 def __hash__ ( self ): 
 return self . _cached_hash 

 def __repr__ ( self ): 
 return ( f " { self . move_type } : (Vertex: { self . vertex } ," 
 f " New_colour: { self . new_colour } , Old_colour: { self . old_colour } )" ) 

 def invert ( self : Move ) -> Move : 
 """ 
 Returns the inverse of the current move. 

 This method constructs and returns the logical inverse of the move. For example: 
 - A `CONSTRUCTION_MOVE` becomes a `DESTRUCTION_MOVE` with the `old_colour` set to the original `new_colour`. 
 - A `DESTRUCTION_MOVE` becomes a `CONSTRUCTION_MOVE` with the `new_colour` set to the original `old_colour`. 
 - A `ONE_EXCHANGE` move swaps the `old_colour` and `new_colour`. 

 Returns 
 ------- 
 Move 
 A new `Move` object representing the inverse of the original move. 
 """ 
 match self . move_type : 
 case Move_type . CONSTRUCTION_MOVE : 
 return Move ( move_type = Move_type . DESTRUCTION_MOVE , vertex = self . vertex , old_colour = self . new_colour ) 
 case Move_type . DESTRUCTION_MOVE : 
 return Move ( move_type = Move_type . CONSTRUCTION_MOVE , vertex = self . vertex , new_colour = self . old_colour ) 
 case Move_type . ONE_EXCHANGE : 
 return Move ( move_type = Move_type . ONE_EXCHANGE , vertex = self . vertex , new_colour = self . old_colour , old_colour = self . new_colour ) 
 case _ : 
 # Can't happen 
 pass

### **Implementation: Graph colouring problem - Solution**
The Solution class implements the SolutionInterface and implements all data necessary to represent a solution, which is the colors of the vertices. To make some operations easier both a dictionary with vertex as key and colour as value and a dictionary of colour as key and the colour class (a set) as value, is implemented. The solution also contains the problem instance and three move generators, one for each type of neighbourghood.

In [ ]:
mo.accordion(
    {
        "View code: ": mo.md(
            f"{
                show_method(
                    graph_colouring,
                    'Solution',
                    '__init__',
                )
            }"
        )
    }
)

class Solution ( problem : 'Problem' ) -> 'None' 
 def __init__ ( self , problem : Problem ) -> None : 
 self . problem = problem 
 self . colour_classes = {} 
 self . vertex_colours = { v : 0 for v in range ( problem . vertex_count )} 
 self . conflicts = [[[ 0 , set ()] for _ in range ( problem . vertex_count + 1 )] for _ in range ( problem . vertex_count )] 

 self . local_move_generator = iter ( utils . sample2 ( self . problem . vertex_count , 
 min ( len ( self . colour_classes . keys ()) + 1 , 
 self . problem . vertex_count ))) 
 self . construction_move_generator = iter ( utils . sample2 ( self . problem . vertex_count , 
 min ( len ( self . colour_classes . keys ()) + 1 , 
 self . problem . vertex_count ))) 
 self . destruction_move_generator = iter ( utils . sample ( self . problem . vertex_count )) 

 def __hash__ ( self ): 
 return self . vertex_colours . __hash__ 

 def __repr__ ( self ): 
 return f " { self . colour_classes } "

The Solution class implements the following methods:

In [ ]:
mo.accordion(
    {
        "**objective_value**: ": mo.vstack(
            [
                mo.md(
                    "As describe in the intro the model uses an evaluation function and not the objective function. However, since evaluation functions isn't supported it is implemented as the objective function. The objective value is calculated as the sum of the square of the size of each colour class, plus the product of number of vertices and the number of conflicts in each colour class."
                ),
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'objective_value',
                        )
                    }"
                ),
            ]
        ),
        "**objective_value_increment**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'objective_value_increment',
                        )
                    }"
                )
            ]
        ),
        "**lower_bound**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'lower_bound',
                        )
                    }"
                )
            ]
        ),
        "**lower_bound_increment**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'lower_bound_increment',
                        )
                    }"
                )
            ]
        ),
        "**apply**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'apply',
                        )
                    }"
                )
            ]
        ),
    }
)

<marimo-accordion data-labels='["<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>objective_value</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>objective_value_increment</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>lower_bound</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>lower_bound_increment</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>apply</strong>: </span></span>"]' data-multiple='false'> As describe in the intro the model uses an evaluation function and not the objective function. However, since evaluation functions isn't supported it is implemented as the objective function. The objective value is calculated as the sum of the square of the size of each colour class, plus the product of number of vertices and the number of conflicts in each colour class. def objective_value ( self ) -> Optional [ float ]: 
 """ 
 Calculates the cost of the solution based on vertex colours and problem constraints. 

 The objective value is computed as the difference between two terms: 
 1. The negative sum of squared sizes of colour classes. 
 2. The sum of twice the product of the size of each colour class and the number of edges between vertices in the class. 

 Parameters 
 ---------- 
 self : Solution 
 The Solution instance for which the objective value is calculated, based on the colouring of the vertices. 

 Returns 
 ------- 
 Optional[float] 
 The objective value as a floating-point number. Returns `None` if the objective cannot be computed. 

 """ 

 # First term: - \sum_{i=1}^{k} |C_i|^2 
 sum_c_squared = sum ( len ( nodes ) ** 2 for nodes in self . colour_classes . values ()) 

 # Second term: \sum_{i=1}^{k} 2|C_i| \cdot |E_i| 
 sum_c_e = 0 
 for vertices in self . colour_classes . values (): 
 edge_count = 0 
 for v in vertices : 
 for u in self . problem . adjacency_list [ v ]: 
 if u in vertices and u < v : 
 edge_count += 1 
 sum_c_e += 2 * len ( vertices ) * edge_count 

 return - sum_c_squared + sum_c_e 
 def objective_value_increment ( self : Solution , move : Move ) -> Optional [ float ]: 
 """ 
 Computes the change in the objective value due to a colour change for a given vertex. 

 Parameters 
 ---------- 
 self : Solution 
 The solution instance in which the objective value change is being calculated. 
 move : Move 
 A `Move` object that contains the information about the vertex whose colour is being changed. It includes 
 the vertex (`vertex`), the old colour (`old_colour`), and the new colour (`new_colour`). 

 Returns 
 ------- 
 Optional[float] 
 The change in the objective value resulting from the colour change. This value is positive or negative, 
 indicating an increase or decrease in the objective value. Returns `None` if the calculation fails. 
 """ 
 # Cache attributes locally (faster than repeated attribute access) 
 vertex = move . vertex 
 old_colour = move . old_colour 
 new_colour = move . new_colour 

 old_class = self . colour_classes [ old_colour ] 
 new_class = self . colour_classes . get ( new_colour , {}) 

 old_size = len ( old_class ) 
 new_size = len ( new_class ) 

 # Term 1: Change in sum of squared sizes 
 delta_size_term = 2 * ( old_size - new_size - 1 ) 

 # Term 2: Added conflicts in colour_class 
 conflict_old = self . conflicts [ vertex ][ old_colour ][ 0 ] 
 conflict_new = self . conflicts [ vertex ][ new_colour ][ 0 ] 

 # Change due to colour_class size difference 
 delta_old_colour_class = self . _total_conflicts_in_colour_class ( old_colour ) 
 delta_new_colour_class = self . _total_conflicts_in_colour_class ( new_colour ) 

 delta_old_edge_term = - 2 * ( old_size * conflict_old + delta_old_colour_class ) 
 delta_new_edge_term = 2 * (( new_size + 1 ) * conflict_new + delta_new_colour_class

There is also implemented the following helper methods:

In [ ]:
mo.accordion(
    {
        "**get_num_colours**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'get_num_colours',
                        )
                    }"
                )
            ]
        ),
        "**adjacent_colours**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            'adjacent_colours',
                        )
                    }"
                )
            ]
        ),
        "**_construction_move**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            '_construction_move',
                        )
                    }"
                )
            ]
        ),
        "**_destruction_move**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            '_destruction_move',
                        )
                    }"
                )
            ]
        ),
        "**_one_exchange**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            '_one_exchange',
                        )
                    }"
                )
            ]
        ),
        "**_swap**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            '_swap',
                        )
                    }"
                )
            ]
        ),
        "**_reset_move_generator**: ": mo.vstack(
            [
                mo.md(
                    f"{
                        show_method(
                            graph_colouring,
                            'Solution',
                            '_reset_move_generator',
                        )
                    }"
                )
            ]
        ),
    }
)

<marimo-accordion data-labels='["<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>get_num_colours</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>adjacent_colours</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>_construction_move</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>_destruction_move</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>_one_exchange</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>_swap</strong>: </span></span>", "<span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>_reset_move_generator</strong>: </span></span>"]' data-multiple='false'> def get_num_colours ( self ) -> int : 
 """ 
 Returns the number of colours used. 

 Parameters 
 ---------- 
 self : Solution 
 The solution instance to which the move is applied. 

 Returns 
 ------- 
 int 
 The number os colours used. 
 """ 
 return len ( self . colour_classes . keys ()) 
 def adjacent_colours ( self , v : int ) -> set [ int ]: 
 """ 
 Returns the set of colours used by vertices adjacent to the given vertex. 

 Parameters 
 ---------- 
 v : int 
 The vertex for which the adjacent colours are to be determined. 

 Returns 
 ------- 
 set[int] 
 A set containing the colours used by adjacent vertices. If no adjacent vertices are assigned a colour, 
 an empty set is returned. 
 """ 
 adjacent_colours : set = set () 
 for u in self . problem . adjacency_list [ v ]: 
 adjacent_colours . add ( self . vertex_colours [ u ]) 
 return adjacent_colours . difference ( * [[ 0 ]]) 
 def _construction_move ( self : Solution , move : Move ) -> Solution : 
 """ 
 Helper method for Apply - Applies a construction move 

 Adds a vertex to the specified colour class in the solution. 

 Parameters 
 ---------- 
 self : Solution 
 The solution instance that is being modified. 
 move : Move 
 A `Move` object containing the vertex (`vertex`) and the new colour (`new_colour`) to be assigned. 

 Returns 
 ------- 
 Solution 
 The updated `Solution` object after the vertex has been added to the specified colour class. 
 """ 
 try : 
 if move . new_colour > len ( self . colour_classes . keys ()): 
 self . colour_classes [ move . new_colour ] = set () 
 self . colour_classes [ move . new_colour ] . add ( move . vertex ) 
 self . vertex_colours [ move . vertex ] = move . new_colour 

 # Update conflict matrix 
 for neighbour in self . problem . adjacency_list [ move . vertex ]: 
 self . conflicts [ neighbour ][ move . new_colour ][ 0 ] += 1 
 self . conflicts [ neighbour ][ move . new_colour ][ 1 ] . add ( move . vertex ) 


 except Exception as e : 
 logging . error ( e ) 
 return self 
 def _destruction_move ( self : Solution , move : Move ) -> Solution : 
 """ 
 Helper method for Apply - Applies a destruction move 

 removes a vertex from the specified colour class in the solution. 

 Parameters 
 ---------- 
 self : Solution 
 The solution instance that is being modified. 
 move : Move 
 A `Move` object containing the vertex (`vertex`) and the old colour (`old_colour`) to be removed. 

 Returns 
 ------- 
 Solution 
 The updated `Solution` object after the vertex has been removed from the specified colour class. 
 """ 
 try : 
 # Update conflict matrix 
 for neighbour in self . problem . adjacency_list [ move . vertex ]: 
 self . conflicts [ neighbour ][ move . old_colour ][ 0 ] -= 1 
 self . conflicts [ neighbour ][ move . old_colour ][ 1 ] . remove ( move . vertex ) 

 self . vertex_colours [ move . vertex ] = 0 
 self . colour_classes [ move . old_colour ] . remove ( move . vertex ) 

 if len ( self . colour_classes [ move . old_colour ]) == 0 : 
 self . _swap ( move . old_colour ) 



 except ValueError :

<br/>
<br/>
<br/>
# Roar-NET API-Interface

In [ ]:
from roar_net_api.data import graph
from roar_net_api import algos
import problem_models

import importlib
import os
import types
import pkgutil


def get_sub_moduels(module):
    return [
        name
        for _, name, _ in pkgutil.iter_modules(
            [os.path.dirname(module.__file__)]
        )
    ]


def setup():
    problem_files = sorted(
        [
            os.path.relpath(
                os.path.join(root, file), os.path.dirname(graph.__file__)
            )
            for root, _, files in os.walk(os.path.dirname(graph.__file__))
            for file in files
        ]
    )
    model_loaders = get_sub_moduels(problem_models)
    solver_functions = get_sub_moduels(algos)
    return problem_files, model_loaders, solver_functions

In [ ]:
problem_files, model_loaders, solver_functions = setup()

problem_instance_dropdown = mo.ui.dropdown(
    problem_files, label="Choose problem instance"
)

model_dropdown = mo.ui.dropdown(model_loaders, label="Choose problem model")

soltion_init_method = mo.ui.dropdown(
    ["Random", "Heuristic", "Empty"], label="Choose soltion initation method"
)

solver_dropdown = mo.ui.dropdown(solver_functions, label="Choose Solver")

layout = mo.callout(
    kind="neutral",
    value=mo.vstack(
        [
            mo.md(
                "**Choose a problem model, problem instance and solver to run:**"
            ),
            problem_instance_dropdown,
            model_dropdown,
            soltion_init_method,
            solver_dropdown,
        ],
        align="center",
    ),
)

#  Render UI
mo.md("### Select Options:")
layout

<marimo-callout-output data-html='"<div style='display: flex;flex: 1;flex-direction: column;justify-content: flex-start;align-items: center;flex-wrap: nowrap;gap: 0.5rem'><div><span class=\"markdown prose dark:prose-invert\"><span class=\"paragraph\"><strong>Choose a problem model, problem instance and solver to run:</strong></span></span></div><div><marimo-ui-element object-id='ulZA-0' random-id='16fdcc79-2ad5-81f0-5ef4-de66c26d0472'><marimo-dropdown data-initial-value='[]' data-label='&quot;&lt;span class=&#92;&quot;markdown prose dark:prose-invert&#92;&quot;&gt;&lt;span class=&#92;&quot;paragraph&#92;&quot;&gt;Choose problem instance&lt;/span&gt;&lt;/span&gt;&quot;' data-options='[&quot;1-FullIns_3.col&quot;, &quot;1-FullIns_4.col&quot;, &quot;1-FullIns_5.col&quot;, &quot;1-Insertions_4.col&quot;, &quot;1-Insertions_5.col&quot;, &quot;1-Insertions_6.col&quot;, &quot;2-FullIns_3.col&quot;, &quot;2-FullIns_4.col&quot;, &quot;2-FullIns_5.col&quot;, &quot;2-Insertions_3.col&quot;, &quot;2-Insertions_4.col&quot;, &quot;2-Insertions_5.col&quot;, &quot;3-FullIns_3.col&quot;, &quot;3-FullIns_4.col&quot;, &quot;3-FullIns_5.col&quot;, &quot;3-Insertions_3.col&quot;, &quot;3-Insertions_4.col&quot;, &quot;3-Insertions_5.col&quot;, &quot;4-FullIns_3.col&quot;, &quot;4-FullIns_4.col&quot;, &quot;4-FullIns_5.col&quot;, &quot;4-Insertions_3.col&quot;, &quot;4-Insertions_4.col&quot;, &quot;5-FullIns_3.col&quot;, &quot;5-FullIns_4.col&quot;, &quot;DSJC1000.1.col&quot;, &quot;DSJC1000.5.col&quot;, &quot;DSJC1000.9.col&quot;, &quot;DSJC125.1.col&quot;, &quot;DSJC125.5.col&quot;, &quot;DSJC125.9.col&quot;, &quot;DSJC250.1.col&quot;, &quot;DSJC250.5.col&quot;, &quot;DSJC250.9.col&quot;, &quot;DSJC500.1.col&quot;, &quot;DSJC500.5.col&quot;, &quot;DSJC500.9.col&quot;, &quot;DSJR500.1.col&quot;, &quot;DSJR500.1c.col&quot;, &quot;DSJR500.5.col&quot;, &quot;__init__.py&quot;, &quot;__pycache__/__init__.cpython-312.pyc&quot;, &quot;abb313GPIA.col&quot;, &quot;anna.col&quot;, &quot;ash331GPIA.col&quot;, &quot;ash608GPIA.col&quot;, &quot;ash958GPIA.col&quot;, &quot;david.col&quot;, &quot;flat1000_50_0.col&quot;, &quot;flat1000_60_0.col&quot;, &quot;flat1000_76_0.col&quot;, &quot;flat300_20_0.col&quot;, &quot;flat300_26_0.col&quot;, &quot;flat300_28_0.col&quot;, &quot;fpsol2.i.1.col&quot;, &quot;fpsol2.i.2.col&quot;, &quot;fpsol2.i.3.col&quot;, &quot;games120.col&quot;, &quot;homer.col&quot;, &quot;huck.col&quot;, &quot;inithx.i.1.col&quot;, &quot;inithx.i.2.col&quot;, &quot;inithx.i.3.col&quot;, &quot;jean.col&quot;, &quot;le450_15a.col&quot;, &quot;le450_15b.col&quot;, &quot;le450_15c.col&quot;, &quot;le450_15d.col&quot;, &quot;le450_25a.col&quot;, &quot;le450_25b.col&quot;, &quot;le450_25c.col&quot;, &quot;le450_25d.col&quot;, &quot;le450_5a.col&quot;, &quot;le450_5b.col&quot;, &quot;le450_5c.col&quot;, &quot;le450_5d.col&quot;, &quot;miles1000.col&quot;, &quot;miles1500.col&quot;, &quot;miles250.col&quot;, &quot;miles500.col&quot;, &quot;miles750.col&quot;, &quot;mug100_1.col&quot;, &quot;mug100_25.col&quot;, &quot;mug88_1.col&quot;, &quot;mug88_25.col&quot;, &quot;mulsol.i.1.col&quot;, &quot;mulsol.i.2.col&quot;, &quot;mulsol.i.3.col&quot;, &quot;mulsol.i.4.col&quot;, &quot;mulsol.i.5.col&quot;, &quot;myciel3.col&quot;, &quot;myciel4.col&quot;, &quot;myciel5.col&quot;, &quot;myciel6.col&quot;, &quot;myciel7.col&quot;, &quot;queen10_10.col&quot;, &quot;queen11_11.col&quot;, &quot;queen12_12.col&quot;, &quot;queen13_13.col&quot;, &quot;queen14_14.col&quot;, &quot;queen15_15.col&quot;, &quot;queen16_16.col&quot;, &quot;queen5_5.col&quot;, &quot;queen6_6.col&quot;, &quot;queen7_7.col&quot;, &quot;queen8_12.col&quot;, &quot;queen8_8.col&quot;, &quot;queen9_9.col&quot;, &quot;r1000.1.col&quot;, &quot;r1000.1c.col&quot;, &quot;r1000.5.col&quot;, &quot;r125.1.col&quot;, &quot;r125.1c.col&quot;, &quot;r125.5.col&quot;, &quot;r250.1.col&quot;, &quot;r250.1c.col&quot;, &quot;r250.5.col&quot;, &quot;school1.col&quot;, &quot;school1_nsh.col&quot;, &quot;wap01a.c

In [ ]:
import math


def get_params(method):
    # Get signature and docstring
    sig = inspect.signature(method)
    docstring = inspect.getdoc(method) or ""

    # Basic param docstring parsing
    param_docs = {}
    for line in docstring.splitlines():
        line = line.strip()
        for name, param in sig.parameters.items():
            if line.startswith(":param " + name):
                param_docs[param] = line[7:]

    # Build UI inputs
    params = []

    for name, param in sig.parameters.items():
        if name in ("self", "cls"):
            continue

        help_text = param_docs.get(param, name)

        if param.annotation == int:
            params.append(
                mo.ui.number(label=help_text, start=-1e6, stop=1e6, value=0)
            )
        elif param.annotation == float:
            params.append(
                mo.ui.number(label=help_text, start=-1e6, stop=1e6, value=0.0)
            )
        elif param.annotation == bool:
            params.append(mo.ui.checkbox(label=help_text))

    return params


if solver_dropdown.value:
    # Load the selected module
    selected_solver_module = importlib.import_module(
        f"roar_net_api.algos.{solver_dropdown.value}"
    )
    solver_options = get_params(
        selected_solver_module.__dict__[solver_dropdown.value]
    )
    solver_options_layout = mo.callout(
        kind="neutral",
        value=mo.vstack(
            [mo.md(f"**Solver options for**: {solver_dropdown.value}")]
            + solver_options,
            align="center",
        ),
    )

In [ ]:
solver_options_layout if solver_dropdown.value else None

In [ ]:
from pathlib import Path
from roar_net_api.api.custom_logger import add_metric_logger
from roar_net_api.api.utils import plot_evaluation_marimo

plot = None


def run(problem, instance_problem_path):
    with mo.status.spinner(title="Loading...") as _spinner:
        _spinner.update("Running")

        # set Plot to True to show plot during run
        metric_log_file_path = add_metric_logger(
            f"runs/metric_logs/{os.path.splitext(os.path.basename(str(problem_instance_dropdown.value)))[0]}.csv",
            False,
        )
        _sol_file_path = Path(
            f"runs/solution_files/{str(metric_log_file_path).rsplit('/', 1)[-1].rsplit('.', 1)[0]}.sol"
        )
        solution = getattr(
            selected_problem_module.Problem,
            soltion_init_method.value.lower() + "_solution",
        )(problem)

        solution = getattr(selected_solver_module, solver_dropdown.value)(
            problem, solution, *[p.value for p in solver_options]
        )

        _spinner.update("Done")

        getattr(selected_problem_module, "gen_sol_file")(
            _sol_file_path, solution
        )

        result = getattr(selected_problem_module, "run_solution_checker")(
            instance_problem_path, _sol_file_path, 1
        )
        text, nodes = getattr(
            selected_problem_module, "check_coloring_constraints"
        )(solution)
        if text:
            print(text)
            for node_pair in nodes:
                print(
                    f"nodes: {node_pair} has colours {getattr(selected_problem_module, 'get_numbers_from_lines')(_sol_file_path, node_pair[0], node_pair[1])}"
                )
        else:
            plot = plot_evaluation_marimo(metric_log_file_path)


problem = None

if (
    problem_instance_dropdown.value
    and model_dropdown.value
    and all(p.value != None for p in solver_options)
    and soltion_init_method.value
):
    # Load the selected module
    selected_problem_module = importlib.import_module(
        f"problem_models.{model_dropdown.value}"
    )

    with open(
        (Path(Path(graph.__file__).parent, problem_instance_dropdown.value)),
        mode="r",
        encoding="utf-8",
    ) as file:
        if problem_instance_dropdown.value.endswith(".txt"):
            problem = selected_problem_module.Problem.from_textio(file)
        elif problem_instance_dropdown.value.endswith(".col"):
            problem = selected_problem_module.Problem.from_col(file)
        else:
            print("Unsupported file format")

In [ ]:
if problem is not None:
    run_layout = mo.vstack(
        [
            mo.ui.button(
                label="Run solver",
                kind="danger",
                on_click=run(
                    problem,
                    problem_instance_dropdown.value,
                ),
            )
        ],
        align="center",
    )
    run_layout